# ETL - Store database

## Goal:
1. Crie o processo de ETL a partir das tabelas fornecidas; modele os dados e explique a metodologia utilizada.


2. Crie dois dashboards usando o Tableau Public a partir do seu modelo de dados, apresentando:

- Análise por lojas, unidades de negócios, canais, produtos, tipos de cliente e período, considerando as medidas: meta, venda, margem; e possíveis métricas como ticket médio por cliente, qtd de items por cupom, etc.

- Análises relevantes que apontem oportunidades e permitam a tomada de decisões estratégicas para o negocio (explique as análises e as conclusões em um storytelling).

## 1. Extraction

Load the files from BigQuery in a dataframe to be manipulated.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import bigquery
import pandas_gbq
import db_dtypes
from tqdm import tqdm

# define style for charts
plt.style.use('ggplot')

# expand number of columns to better viz
pd.set_option('display.max_columns', 50)

In [ ]:
# use local json file to authenticate
SERVICE_ACCOUNT_JSON = r"D:\DataAnalytics\projects\sales-store-383520-cd6fdfbcbcb6.json"
client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)

# find all tables in dataset (raw)
tables = client.list_tables("sales-store-383520.raw")

# iterate through tables and create a dataframe for each table
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

    query = 'select * from `sales-store-383520.raw.{}`'.format(table.table_id)

    globals()[f"df_{table.table_id}"] = pandas_gbq.read_gbq(query, project_id='sales-store-383520', progress_bar_type='tqdm_notebook')

## 2. Transforming
Performe some adjustments on imported data.

### 2.1 Normalizing the column names

### 2.2 Change datatypes


#### 2.2.1 Change the columns related to date, originally as string

In [4]:
def change_to_date(df, cols):
    for c in cols:
        df[c] = pd.to_datetime(df[c], format="%Y/%m/%d")

change_to_date(df_sales, ['date'])

change_to_date(df_business_goal, ['date' ])


#### 2.2.2 Change the columns related to numbers to float, originally as string.

In [5]:
def change_to_float(df, cols):
    for c in cols:
        df[c] = df[c].str.replace(',', '.').astype(float)

change_to_float(df_sales, ['items', 'gross_revenue', 'tax_value', 'costs'])

change_to_float(df_business_goal, ['sales_goal' ])

#### 2.2.3 Removing null or columns with 0 items of Sales dataframe

In [6]:
print("Before: {}".format(df_sales[df_sales['items'] == 0]['items'].count()))

df_sales = df_sales[df_sales['items'] > 0]

print("After: {}".format(df_sales[df_sales['items'] == 0]['items'].count()))

Before: 11397
After: 0


### 2.3 Creating new columns to be analized/ plotted.

#### 2.3.1 Creating the tax rate column to understand the percentagem of taxes of each sale.


In [7]:
df_sales['tax_rate'] = round(df_sales['tax_value'] / df_sales['gross_revenue'], 4)

df_sales[['gross_revenue', 'tax_value', 'tax_rate']].sample(5)

,gross_revenue,tax_value,tax_rate
2761517,3.912,1.062,0.2715
196584,6.912,0.642,0.0929
3127055,104.994,9.714,0.0925
689467,4.536,0.420,0.0926
1573156,5.994,1.632,0.2723


#### 2.3.2 Create a net revenue column

In [8]:
df_sales['net_revenue'] = df_sales['gross_revenue'] - (df_sales['tax_value'] + df_sales['costs'])

df_sales[['gross_revenue', 'net_revenue', 'tax_value' , 'costs']].sample(5)

,gross_revenue,net_revenue,tax_value,costs
1977553,38.940,10.080,8.778,20.082
1662710,1.674,0.438,0.456,0.780
2927271,7.194,2.226,1.962,3.006
1539576,3.594,1.356,0.330,1.908
312568,27.588,9.618,7.518,10.452


### 2.3.3 New column with cumulative sales for each customer

In [9]:
df_sales['cumulative_sales'] = df_sales.assign(temp=~df_sales.duplicated(subset=['id_customer','date'])).groupby('id_customer')['temp'].cumsum()

### 2.4 Create new dataframe 'Customers' to be used in analysis and data visualization. 

#### 2.4.1 Create the new dataframe using information of other tables

In [10]:
# get the customer information in the sales dataframe
df_customers = df_sales.groupby(['id_customer', 'id_customer_type'])\
                                        .agg({'id_customer': 'count', 'items': 'sum','date': ['min', 'max'], 'gross_revenue': 'sum'})\
                                        .reset_index()

# rename columns
df_customers.columns = ['id_customer', 'id_customer_type', 'purchase_count', 'items_purchased', 'first_purchase', 'last_purchase', 'total_spent']

# get the customer type information
df_customers = df_customers.merge(df_customer_type, on='id_customer_type', how='left')

# create a column to identify the customer type
df_customers['customer_type_code'] = df_customers.apply(lambda x: 1 if x['customer_type'] == 'Identificado' else 0, axis=1)

df_customers.sample(5)

,id_customer,id_customer_type,purchase_count,items_purchased,first_purchase,last_purchase,total_spent,customer_type,customer_type_code
20631,BLR6-02Q)%I26WVZM7G\C0,F+9C/:YY=_[^&$L90;9D_%,2,6.0,2021-08-12,2021-08-12,16.020,Identificado,1
112210,J1F9I*N[Q0A3UUWP!&+8M*,"N3ZH'W$AE#+&45Z8N8""S*#",2,1.2,2022-05-19,2022-08-18,24.588,Não Identificado,0
131033,"KN[,.>#`MS_+$""X#A/00!""","N3ZH'W$AE#+&45Z8N8""S*#",1,0.6,2022-08-17,2022-08-17,112.854,Não Identificado,0
106540,ITFOWB-[ZBR4[]-_8?(6H&,F+9C/:YY=_[^&$L90;9D_%,1,0.6,2022-01-20,2022-01-20,42.594,Identificado,1
115473,JA:;G:[-C13(N[7++6W5W-,"N3ZH'W$AE#+&45Z8N8""S*#",20,14.4,2022-09-18,2022-12-20,833.436,Não Identificado,0


#### 2.4.2 Agreggate information by customer.

In [11]:
df_customers = df_customers.groupby('id_customer')\
                                        .agg({'purchase_count': 'sum', 
                                              'items_purchased': 'sum',
                                              'first_purchase': 'min', 
                                              'last_purchase': 'max', 
                                              'total_spent': 'sum', 'customer_type_code': 'max'})\
                                        .reset_index()
df_customers.sample(5)

,id_customer,purchase_count,items_purchased,first_purchase,last_purchase,total_spent,customer_type_code
38589,D;96J1?$3T>8M*.Q?9Z4(+,45,30.0,2021-04-07,2022-08-24,534.246,0
98584,IJQ(:\#9$+Q4TM'[EAU;00,4,4.2,2021-06-08,2021-06-27,62.898,0
140717,"M+LO/QT8<%""3M7)$-O4F$%",10,6.6,2021-12-22,2022-12-17,122.334,1
82453,"H'Q,J/""7>?G)CTQ5>?@02'",22,15.0,2021-01-29,2022-10-25,466.338,0
24060,"C'D-`ZDX0)CF,-@[:,0J/'",1,0.6,2021-06-08,2021-06-08,11.994,0


#### 2.4.3 Get the Customer state from address customers table

In [12]:
temp_state_customer = df_sales.merge(df_address_customers, on='id_address_sale', how='left')

temp_state_customer[['id_customer', 'customer_state']]

df_customers = df_customers.merge(temp_state_customer[['id_customer', 'customer_state']], on='id_customer', how='left')

df_customers.sample(5)

,id_customer,purchase_count,items_purchased,first_purchase,last_purchase,total_spent,customer_type_code,customer_state
124422,AEA`R/3!J<5NW+8X.#$1X),110,76.2,2021-01-22,2022-12-27,2.562708e+03,0,SP
2818914,"M]X;GRX3YFUIIRU_,;:AB+",678206,527065.8,2021-01-02,2022-12-31,1.264691e+07,0,NaN
1295182,"G%FBJ3?[6Y@&WQ/$8A,(%%",6,3.6,2021-04-25,2021-05-16,7.826400e+01,0,SP
624065,"D""%A*B6?.8LZ0UK[Z;F$('",21,13.8,2021-12-30,2022-12-16,3.971460e+02,0,SP
1635860,"HUY)DF=RN$PK3VAB`<""UT%",45,27.0,2021-03-17,2022-12-18,5.463600e+02,0,SP


### 2.5 Aggregate data to Sales dataframe

New: df_sales
- Sales information from "df_sales"
- Business unit information from "df_business_unit"
- Channel information from "df_channel"

In [13]:
df_sales = df_sales.merge(df_business_unit, on='id_business_unit', how='left')

df_sales = df_sales.merge(df_channel, on='id_channel', how='left')

df_sales = df_sales.merge(df_customer_type, on='id_customer_type', how='left')

In [14]:
df_sales = df_sales[['date', 
                     'id_store', 'business_unit', 
                     'channel', 'id_product',
                     'id_customer', 'customer_type',
                     'id_coupon', 'id_address_sale', 
                     'items', 'gross_revenue', 'tax_value', 'costs', 'tax_rate','net_revenue', 
                     'cumulative_sales']]

df_sales

,date,id_store,business_unit,channel,id_product,id_customer,customer_type,id_coupon,id_address_sale,items,gross_revenue,tax_value,costs,tax_rate,net_revenue,cumulative_sales
0,2022-02-23,F)T`P;^+F]5F7YX^S\=+?&,Produtos,Loja,A!&M-NL@E_R<\TCI3*6%C(,AC'3ZC0=Z`I2&`*(:*N<T!,Não Identificado,"H1+4Q;=5-,@]!+@3S1_$J#",D`.@?7[^TKOHZY4J#R+H[',0.6,59.994,16.350,20.610,0.2725,23.034,1
1,2021-10-12,"F%#+YX,X!FRF<FHD):`=9+",Produtos,Loja,"A!VJA!QEMBB<\`B^/,KKI%","M]X;GRX3YFUIIRU_,;:AB+",Não Identificado,"L/R1GFK+5]YTG,H7!S""/M#","AZ4F/""B\;M^HX*YO33V-B&",0.6,35.994,9.810,11.682,0.2725,14.502,1
2,2021-01-13,"F%#+YX,X!FRF<FHD):`=9+",Produtos,Loja,A!^:UL>IR9I<\&W-'JMTQ*,CI9%JME#Q;E/5(07%M2(\.,Não Identificado,"KN(^KDI5K""`S70O>TCKX&*","PH/E(5N""7J*RG[#9LUQ>;$",0.6,89.994,24.528,21.198,0.2726,44.268,1
3,2021-10-25,"F%#+YX,X!FRF<FHD):`=9+",Produtos,Loja,"A!^P6LFR+1:<""FF\9:G,X0",D?4-X5F3-B-Q9%SWI`#79%,Não Identificado,"M'O3PY/2O]AX/RHH/""0PQ(",N2D@8]7!=DPQHM44^.@\1.,0.6,7.200,1.962,2.382,0.2725,2.856,1
4,2022-06-29,F)T`P;^+F]5F7YX^S\=+?&,Produtos,Loja,"A!^P6LFR+1:<""FF\9:G,X0",L;J_H@23VAJ*]>^UPYX(Y+,Identificado,"M-R9S9,NKMI:)Z\1;F_@0&","H$/OM?:%;4HN6R""\8,8C+%",0.6,8.340,2.274,2.382,0.2727,3.684,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4048284,2022-04-18,"FO1G""YC0G6I&C(,H&(MT3-",Produtos,Digital,C#K_]W/^W15=C!+ZT.TO(&,P`-QHBN1%&QH);^318P!Q+,Não Identificado,IMU@RZ#JUH/B[3J8=6^R*%,"L9,'\Q'[KLW1=QX?^H$[[-",0.6,66.474,6.150,36.876,0.0925,23.448,16
4048285,2021-09-06,"FO1G""YC0G6I&C(,H&(MT3-",Produtos,Loja,F2=;Y[92=%93L\GN8O?6/0,P`JZ;'&T^V?>T]EG2FE$F-,Não Identificado,"OD+C04#E@^@E?@@[Z_,KG*","D^C;0^A)C^%U.I<;8."";""'",4.8,14.910,4.062,9.024,0.2724,1.824,5
4048286,2022-02-06,"FO1G""YC0G6I&C(,H&(MT3-",Produtos,Loja,FY=*X['+Y\S(Q/>9IQ.BE/,P`N94US[:Q'MI[3B#_H\X*,Não Identificado,"M39`Q,NY1*H-%%%$B6;-7*","NOT45P>5WJRMF`&ASBF7;""",0.6,6.300,0.588,2.796,0.0933,2.916,2
4048287,2022-08-16,"FO1G""YC0G6I&C(,H&(MT3-",Produtos,Loja,PQC*H<SR1%J]PEP4<P?6`%,P`TO]@HY#)H9$$2X$X8<?$,Não Identificado,"NO25WDDT9H31@:\I#3?O""%",LG'W'KS31A@L.N$?D;$()/,0.6,38.994,7.020,15.438,0.1800,16.536,1


### 2.6 Agregate to Business Goals dataframe more information.

#### 2.6.1 Get the sales of each day, store and branch

In [15]:
sales_period_store = df_sales.groupby(['date', 'id_store'])\
                .agg({
                    'items': 'sum',
                    'gross_revenue': 'sum'}).reset_index()

sales_period_store

,date,id_store,items,gross_revenue
0,2021-01-02,F!25!6;D=./F%2(E)D;]P0,1119.6,25898.100
1,2021-01-02,"F%#+YX,X!FRF<FHD):`=9+",907.8,23662.638
2,2021-01-02,F)T`P;^+F]5F7YX^S\=+?&,1154.4,30992.874
3,2021-01-02,"FO)5JW59TP?&C:?,ZG$$L*",916.2,19486.866
4,2021-01-03,F!25!6;D=./F%2(E)D;]P0,878.4,23619.408
...,...,...,...,...
3569,2022-12-31,F!25!6;D=./F%2(E)D;]P0,706.8,19767.834
3570,2022-12-31,"F%#+YX,X!FRF<FHD):`=9+",699.0,18471.312
3571,2022-12-31,F)T`P;^+F]5F7YX^S\=+?&,1224.6,35505.378
3572,2022-12-31,"FO)5JW59TP?&C:?,ZG$$L*",844.2,18085.818


### 2.6.2 Get the goal for each day, store

In [16]:
goal_period_store = df_business_goal.groupby(['date', 'id_store'])\
                .agg({
                    'sales_goal': 'sum' }).reset_index()

goal_period_store

,date,id_store,sales_goal
0,2021-01-01,F!25!6;D=./F%2(E)D;]P0,0.000
1,2021-01-01,"F%#+YX,X!FRF<FHD):`=9+",0.000
2,2021-01-01,F)T`P;^+F]5F7YX^S\=+?&,0.000
3,2021-01-01,"FO)5JW59TP?&C:?,ZG$$L*",0.000
4,2021-01-02,F!25!6;D=./F%2(E)D;]P0,22445.046
...,...,...,...
3614,2022-12-31,F!25!6;D=./F%2(E)D;]P0,18323.364
3615,2022-12-31,"F%#+YX,X!FRF<FHD):`=9+",16336.596
3616,2022-12-31,F)T`P;^+F]5F7YX^S\=+?&,33015.048
3617,2022-12-31,"FO)5JW59TP?&C:?,ZG$$L*",19925.220


#### 2.6.3 Merge the Goal per day and store with Sales per day and store.

In [17]:
df_goal_store_period = goal_period_store.merge(sales_period_store, on=['id_store', 'date'], how='left')

# drop the days without goal
df_goal_store_period = df_goal_store_period.dropna(subset=['sales_goal', 'gross_revenue'])

df_goal_store_period

,date,id_store,sales_goal,items,gross_revenue
4,2021-01-02,F!25!6;D=./F%2(E)D;]P0,22445.046,1119.6,25898.100
5,2021-01-02,"F%#+YX,X!FRF<FHD):`=9+",19573.326,907.8,23662.638
6,2021-01-02,F)T`P;^+F]5F7YX^S\=+?&,25188.930,1154.4,30992.874
7,2021-01-02,"FO)5JW59TP?&C:?,ZG$$L*",17902.728,916.2,19486.866
8,2021-01-03,F!25!6;D=./F%2(E)D;]P0,22445.046,878.4,23619.408
...,...,...,...,...,...
3614,2022-12-31,F!25!6;D=./F%2(E)D;]P0,18323.364,706.8,19767.834
3615,2022-12-31,"F%#+YX,X!FRF<FHD):`=9+",16336.596,699.0,18471.312
3616,2022-12-31,F)T`P;^+F]5F7YX^S\=+?&,33015.048,1224.6,35505.378
3617,2022-12-31,"FO)5JW59TP?&C:?,ZG$$L*",19925.220,844.2,18085.818


#### 2.6.4 Create new columns to be used futher.

In [18]:
df_goal_store_period['goal_%'] = round((df_goal_store_period['gross_revenue'] / df_goal_store_period['sales_goal']), 2)

df_goal_store_period['goal_result'] = df_goal_store_period.apply(lambda x: 1 if x['goal_%'] >= 1 else 0, axis=1)

df_goal_store_period

,date,id_store,sales_goal,items,gross_revenue,goal_%,goal_result
4,2021-01-02,F!25!6;D=./F%2(E)D;]P0,22445.046,1119.6,25898.100,1.15,1
5,2021-01-02,"F%#+YX,X!FRF<FHD):`=9+",19573.326,907.8,23662.638,1.21,1
6,2021-01-02,F)T`P;^+F]5F7YX^S\=+?&,25188.930,1154.4,30992.874,1.23,1
7,2021-01-02,"FO)5JW59TP?&C:?,ZG$$L*",17902.728,916.2,19486.866,1.09,1
8,2021-01-03,F!25!6;D=./F%2(E)D;]P0,22445.046,878.4,23619.408,1.05,1
...,...,...,...,...,...,...,...
3614,2022-12-31,F!25!6;D=./F%2(E)D;]P0,18323.364,706.8,19767.834,1.08,1
3615,2022-12-31,"F%#+YX,X!FRF<FHD):`=9+",16336.596,699.0,18471.312,1.13,1
3616,2022-12-31,F)T`P;^+F]5F7YX^S\=+?&,33015.048,1224.6,35505.378,1.08,1
3617,2022-12-31,"FO)5JW59TP?&C:?,ZG$$L*",19925.220,844.2,18085.818,0.91,0


## 3. Load data

After manipulate the data we are ready to send the datframe to in Tableau to analysis and data visualization.

In [19]:
def send_to_bq(df, table_name):
    pandas_gbq.to_gbq(df, table_name, project_id='sales-store-383520', if_exists='replace')

In [22]:
send_to_bq(df_products, 'data.products')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [23]:
send_to_bq(df_stores, 'data.stores')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [24]:
send_to_bq(df_sales, 'data.sales')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [25]:
send_to_bq(df_customers, 'data.customers')

100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]


In [26]:
send_to_bq(df_goal_store_period, 'data.business_goals')

100%|██████████| 1/1 [00:00<?, ?it/s]


Save the CSV locally if needed.

In [24]:
df_sales.to_csv('data_load/sales.csv', index=None)

df_customers.to_csv('data_load/customers.csv', index=None)

df_goal_store_period.to_csv('data_load/business_goals.csv', index=None)

df_stores.to_csv('data_load/stores.csv', index=None)

df_products.to_csv('data_load/products.csv', index=None)

## Next Steps

1. Automate data extraction
2. Fine tune to performe faster
3. Run it periodically